# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: **Triệu Nhật Minh**
- MSSV: **21127112**
- Lớp: **21CLC02**

# Import

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Latex
from sklearn.model_selection import KFold
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [92]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Tạo dictionary chứa tên các cột

In [93]:
dict_column = {1: 'Gender', 2: '10percentage', 3: '12percentage', 4: 'CollegeTier', 5: 'Degree', 6: 'collegeGPA', 7: 'CollegeCityTier', 8: 'English', 9: 'Logical', 10: 'Quant', 11: 'Domain', 12: 'ComputerProgramming', 13: 'ElectronicsAndSemicon', 14: 'ComputerScience', 15: 'MechanicalEngg', 16: 'ElectricalEngg', 17: 'TelecomEngg', 18: 'CivilEngg', 19: 'conscientiousness', 20: 'agreeableness', 21: 'extraversion', 22: 'nueroticism', 23: 'openess_to_experience', 24: 'Salary'}

# Cài đặt hàm

In [94]:
# Cài đặt các hàm cần thiết ở đây

In [95]:
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.pinv(X) # X_pinv = np.linalg.inv(X.T @ X) @ X.T 
        self.w = X_pinv @ y
        return self

    def get_params(self):
        return self.w.round(3)

    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)   

In [96]:
def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))

In [97]:
def latex_text(params):
    text = '$$\\text{Salary} = ' + str(params[0][0]) + ' \\times \\text{' + dict_column[1] + '}'
    for i in range(1, len(params)):
        text += str(params[i][0]) if params[i][0] < 0 else ' + ' + str(params[i][0])
        text += ' \\times \\text{' + dict_column[i+1] + '}'
        if i % 4 == 0:
            text += ' \\\\ '
    text += '$$'
    return text

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [98]:
# Phần code cho yêu cầu 1a
X_1a = train.iloc[:, :11]
y_1a = train.iloc[:, -1]

X_1a_test = test.iloc[:, :11]
y_1a_test = test.iloc[:, -1]

lr = OLSLinearRegression().fit(X_1a, y_1a)

params = lr.get_params().reshape(-1,1)

y_pred_1a = lr.predict(X_1a_test)

In [99]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae_1a = mae(y_test, y_pred_1a)
mae_1a

104863.7775403313

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

In [100]:
Latex(latex_text(params))

<IPython.core.display.Latex object>

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [101]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

# Chọn ra 5 đặc trưng (5 cột cuối tập X_train)
X_1b = X_train.iloc[:, -5:]

# Shuffle k = 20 (k-fold cross validation)
k = 20

### **conscientiousness**

In [102]:
X_1b_cons = X_1b['conscientiousness']
kf = KFold(n_splits=k, shuffle=True, random_state=42)
MAE_cons = []
for train_index in kf.split(X_1b_cons):
    X_train = X_1b_cons[train_index]
    display(X_train)
    # lr = OLSLinearRegression().fit(
    # X_test = X_1b_cons[test_index]

    # y_pred = lr.predict(X_test)
    # MAE_cons.append(mae(y_test, y_pred))

KeyError: 'key of type tuple not found and not a MultiIndex'

In [ ]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = \omega_1 \times \text{agreeableness}$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [ ]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

In [ ]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [ ]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [ ]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

In [ ]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện

In [ ]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = ...$$